In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.datasets import mnist
import keras_tuner as kt

In [19]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Flatten the images for the ANN
x_train = x_train.reshape(x_train.shape[0], 28 * 28)
x_test = x_test.reshape(x_test.shape[0], 28 * 28)

In [22]:
# Build a function to create the model for Keras Tuner
def build_model(hp):
    model = Sequential([
        Dense(hp.Int('units_1', min_value=64, max_value=512, step=64), 
              activation='relu', kernel_regularizer=l2(0.01), input_shape=(28 * 28,)),
        Dropout(hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)),
        Dense(hp.Int('units_2', min_value=64, max_value=512, step=64), activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1)),
        Dense(10, activation='softmax')  # 10 classes for MNIST digits
    ])
    
    # Choose the optimizer and learning rate
    optimizer = hp.Choice('optimizer', values=['adam', 'sgd'])
    if optimizer == 'adam':
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG')),
                      loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    else:
        model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG')),
                      loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [25]:
# Instantiate the Keras Tuner RandomSearch object
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # You can increase this to explore more combinations
    executions_per_trial=3,  # Number of models to train for each trial
    directory='keras_tuner',
    project_name='mnist_ANN'
)

Reloading Tuner from keras_tuner\mnist_ANN\tuner0.json


In [30]:
# Set up early stopping to avoid overfitting during hyperparameter search
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Perform the hyperparameter search
tuner.search(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stopping_callback])

# Get the best model and its hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters: ")
print(f"Units in Layer 1: {best_hyperparameters.get('units_1')}")
print(f"Units in Layer 2: {best_hyperparameters.get('units_2')}")
print(f"Dropout 1: {best_hyperparameters.get('dropout_1')}")
print(f"Dropout 2: {best_hyperparameters.get('dropout_2')}")
print(f"Optimizer: {best_hyperparameters.get('optimizer')}")
print(f"Learning rate: {best_hyperparameters.get('learning_rate')}")

# Evaluate the best model on the test set
test_loss, test_accuracy = best_model.evaluate(x_test, y_test)
print(f'Test accuracy of the best model: {test_accuracy}')

Best Hyperparameters: 
Units in Layer 1: 128
Units in Layer 2: 512
Dropout 1: 0.4
Dropout 2: 0.2
Optimizer: adam
Learning rate: 0.00012532888305788302
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9565 - loss: 0.2955
Test accuracy of the best model: 0.9643999934196472
